In [1]:
]st

    Status `/mnt/E4E0A9C0E0A998F6/github/ReinforcementLearningAnIntroduction.jl/notebooks/Project.toml`
  [02c1da58] RLIntro v0.2.0 [`..`]
  [158674fc] ReinforcementLearning v0.4.0 [`../../ReinforcementLearning.jl`]
  [25e41dd2] ReinforcementLearningEnvironments v0.1.1


In [2]:
using ReinforcementLearning, ReinforcementLearningEnvironments, RLIntro
using RLIntro.TicTacToe

env = TicTacToeEnv()

┌ Info: Precompiling ReinforcementLearning [158674fc-8238-5cab-b5ba-03dfc80d1318]
└ @ Base loading.jl:1264


___
___
___
isdone = [false], winner = [nothing]


In [3]:
nstates, nactions = length(observation_space(env)), length(action_space(env))

(5478, 10)

If you are curious why there are `5478` states, you may see the discussions [here](https://math.stackexchange.com/questions/485752/tictactoe-state-space-choose-calculation/485852)

In [4]:
observe(env)

Observation{Float64,Bool,Int64,NamedTuple{(:legal_actions,),Tuple{Array{Bool,1}}}}(0.0, false, 4175, (legal_actions = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],))

Now we'll use the Monte Carlo based method to estimate the value of each state for each player. Think about this, if we have the precise estimation of each state after taking some specific observation according to current observation, then we can just choose the action that leads to the maximum estimation.

Let's create a value approximator first (here we use the `TabularVApproximator` defined in `ReinforcementLearning.jl`):

In [5]:
V1 = TabularVApproximator(nstates)
V2 = TabularVApproximator(nstates)

TabularVApproximator([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

As you can see, by default all the estimations are initialed with `0.0`. Usually it won't be a problem, but here we can initialize it with a better starting point. For each state, we can check that if the state is a final state and set the initial estimation accordingly.

In [6]:
function init_V!(V, role)
    for i in 1:length(V.table)
        s = TicTacToe.ID2STATE[i]
        isdone, winner = TicTacToe.STATES_INFO[s]
        if isdone
            if winner === nothing
                V.table[i] = 0.5
            elseif winner === role
                V.table[i] = 1.
            else
                V.table[i] = 0.
            end
        else
            V.table[i] = 0.5
        end
    end
    V
end

init_V! (generic function with 1 method)

In [7]:
init_V!(V1, TicTacToe.offensive)
init_V!(V2, TicTacToe.defensive)

TabularVApproximator([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

Then we construct a `MonteCarloLearner` for each player. Here the `MonteCarloLearner` is just a wrapper around the approximator.

In [8]:
learner_1 = MonteCarloLearner(V1; α=0.1, kind=:EveryVisit)
learner_2 = MonteCarloLearner(V2; α=0.1, kind=:EveryVisit)

MonteCarloLearner{:EveryVisit,TabularVApproximator,CachedSampleAvg}(TabularVApproximator([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), 1.0, 0.1, CachedSampleAvg(Dict{Any,Any}()))

Finally we will create the `MonteCarloAgent`. To create such an agent, we need to provide a `learner` and a `policy`. We already have the learners above. Now let's create a policy.

A policy is a mapping from states to actions. Considering that we already have the estimations of states, a simple policy would be checking the estimation of the following up states and select one action which will result to the best state.

In [9]:
function create_policy(V, role, ϵ=0.01)
    obs -> begin
        legal_actions, state = findall(get_legal_actions(obs)), get_state(obs)
        next_states = TicTacToe.get_next_states(TicTacToe.ID2STATE[state], role, legal_actions)
        next_state_estimations = [V(TicTacToe.STATE2ID[ns]) for ns in next_states]
        max_val, idx = findmax(next_state_estimations)
        rand() < ϵ ? rand(legal_actions) : legal_actions[idx]
    end
end

create_policy (generic function with 2 methods)

In [10]:
π_1 = create_policy(V1, TicTacToe.offensive)
π_2 = create_policy(V2, TicTacToe.defensive)

#7 (generic function with 1 method)

In [11]:
agent_1 = MonteCarloAgent(learner_1, π_1, episode_RTSA_buffer();role=TicTacToe.offensive);
agent_2 = MonteCarloAgent(learner_2, π_2, episode_RTSA_buffer();role=TicTacToe.defensive);

MonteCarloAgent{MonteCarloLearner{:EveryVisit,TabularVApproximator,CachedSampleAvg},var"##7#9"{TabularVApproximator,RLIntro.TicTacToe.Defensive,Float64},EpisodeTurnBuffer{(:reward, :terminal, :state, :action),Tuple{Float64,Bool,Int64,Int64},NamedTuple{(:reward, :terminal, :state, :action),Tuple{Array{Float64,1},Array{Bool,1},Array{Int64,1},Array{Int64,1}}}},RLIntro.TicTacToe.Defensive,ReinforcementLearning.TrainingMode}(O, ReinforcementLearning.TrainingMode(), MonteCarloLearner{:EveryVisit,TabularVApproximator,CachedSampleAvg}(TabularVApproximator([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), 1.0, 0.1, CachedSampleAvg(Dict{Any,Any}())), var"##7#9"{TabularVApproximator,RLIntro.TicTacToe.Defensive,Float64}(TabularVApproximator([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), O, 0.01), NamedTuple{(:reward, :terminal, :state, :action),Tuple{Float64,Bool,Int64,Int64}}[])

In [12]:
run((agent_1, agent_2), env, StopAfterStep(1000000; is_show_progress=false))

2-element Array{EmptyHook,1}:
 EmptyHook()
 EmptyHook()

In [13]:
eval_agent_1 = similar(agent_1; mode = RL.EVALUATING_MODE, π=create_policy(V1, agent_1.role, 0.0));
eval_agent_2 = similar(agent_2; mode = RL.EVALUATING_MODE, π=create_policy(V2, agent_2.role, 0.0));

MonteCarloAgent{MonteCarloLearner{:EveryVisit,TabularVApproximator,CachedSampleAvg},var"##7#9"{TabularVApproximator,RLIntro.TicTacToe.Defensive,Float64},EpisodeTurnBuffer{(:reward, :terminal, :state, :action),NTuple{4,Tuple{Float64,Bool,Int64,Int64}},NamedTuple{(:reward, :terminal, :state, :action),NTuple{4,Array{Tuple{Float64,Bool,Int64,Int64},1}}}},RLIntro.TicTacToe.Defensive,ReinforcementLearning.EvaluatingMode}(O, ReinforcementLearning.EvaluatingMode(), MonteCarloLearner{:EveryVisit,TabularVApproximator,CachedSampleAvg}(TabularVApproximator([0.5, 0.55, 0.5, 0.5, 0.0018793574018245605, 0.5, 0.5, 0.5, 0.55, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]), 1.0, 0.1, CachedSampleAvg(Dict{Any,Any}(4460 => SampleAvg(335, 0.5179104477611935),1131 => SampleAvg(34, 0.0),3532 => SampleAvg(234, 0.4914529914529915),2126 => SampleAvg(2, 1.0),4809 => SampleAvg(7, 0.4285714285714286),1461 => SampleAvg(70, 0.49999999999999983),3335 => SampleAvg(85, 0.0),4156 => SampleAvg(2, 0.0),1124 =>

In [15]:
hook = (TotalRewardPerEpisode(;tag="Evaluating"), TotalRewardPerEpisode(;tag="Evaluating"))

run((eval_agent_1, eval_agent_2), env, StopAfterEpisode(1);hook=hook)  # the policy is deterministic now, so one episode will be enough.

sum(hook[1].rewards .== 0.5), sum(hook[2].rewards .== 0.5)

(1, 1)

Now it's your turn to play this game!

In [26]:
function read_action_from_stdin()
    print("Your input:")
    input = parse(Int, readline())
    !in(input, 1:9) && error("invalid input!")
    input
end

function play()
    env = TicTacToeEnv()
    println("""You play first!
    1 4 7
    2 5 8
    3 6 9""")
    while true
        action = read_action_from_stdin()
        env(action)
        println(env)
        obs = observe(env, TicTacToe.offensive)
        if get_terminal(obs)
            if get_reward(obs) == 0.5
                println("Tie!")
            elseif get_reward(obs) == 1.0 
                println("You win!")
            else
                println("Invalid input!")
            end
            break
        end

        env(eval_agent_2(observe(env)))
        println(env)
        obs = observe(env, TicTacToe.defensive)
        if get_terminal(obs)
            if get_reward(obs) == 0.5
                println("Tie!")
            elseif get_reward(obs) == 1.0 
                println("Your lose!")
            else
                println("You win!")
            end
            break
        end
    end
end

play (generic function with 1 method)

In [27]:
play()

You play first!
1 4 7
2 5 8
3 6 9
Your input:stdin> 3
___
___
X__
isdone = [false], winner = [nothing]

___
_O_
X__
isdone = [false], winner = [nothing]

Your input:stdin> 9
___
_O_
X_X
isdone = [false], winner = [nothing]

___
_O_
XOX
isdone = [false], winner = [nothing]

Your input:stdin> 2
___
XO_
XOX
isdone = [false], winner = [nothing]

_O_
XO_
XOX
isdone = [true], winner = [O]

Your lose!
